## Packages

In [73]:
import pandas as pd
from scipy import stats

## Import Data

In [7]:
data_1 = pd.read_csv('data_1.csv')
data_2 = pd.read_csv('data_2.csv')
data_3 = pd.read_csv('data_3.csv')
data = pd.concat([data_1, data_2, data_3], ignore_index=True)

/var/folders/60/75vh_thj2m57nynzg5tgw0300000gn/T/ipykernel_79963/3829154129.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data_1 = pd.read_csv('data_1.csv')
/var/folders/60/75vh_thj2m57nynzg5tgw0300000gn/T/ipykernel_79963/3829154129.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2 = pd.read_csv('data_2.csv')
/var/folders/60/75vh_thj2m57nynzg5tgw0300000gn/T/ipykernel_79963/3829154129.py:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data_3 = pd.read_csv('data_3.csv')


In [9]:
data

,PERMNO,date,TICKER,COMNAM,PERMCO,PRC,VOL,RET,RETX
0,10158,2014-01-02,AMRC,AMERESCO INC,53454,9.78000,73300.0,0.012422,0.012422
1,10158,2014-01-03,AMRC,AMERESCO INC,53454,9.94000,105100.0,0.016360,0.016360
2,10158,2014-01-06,AMRC,AMERESCO INC,53454,9.74000,76400.0,-0.020121,-0.020121
3,10158,2014-01-07,AMRC,AMERESCO INC,53454,9.95000,86900.0,0.021561,0.021561
4,10158,2014-01-08,AMRC,AMERESCO INC,53454,10.12000,79900.0,0.017085,0.017085
...,...,...,...,...,...,...,...,...,...
5715131,93436,2023-12-22,TSLA,TESLA INC,53453,252.53999,93148541.0,-0.007701,-0.007701
5715132,93436,2023-12-26,TSLA,TESLA INC,53453,256.60999,86700724.0,0.016116,0.016116
5715133,93436,2023-12-27,TSLA,TESLA INC,53453,261.44000,106250779.0,0.018822,0.018822
5715134,93436,2023-12-28,TSLA,TESLA INC,53453,253.17999,113250680.0,-0.031594,-0.031594


## Cumulative Return (top 25% are winners and last 25% are losers)

Data Source: https://www.ishares.com/us/products/239714/ishares-russell-3000-etf

In [15]:
r3000 = pd.read_excel('Russell3000.xlsx', sheet_name='Holdings')
r3000

,Ticker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Quantity,Price,Location,Exchange,Currency,FX Rate,Accrual Date
0,--,EMPIRE RESORT ESCROW,Consumer Discretionary,Equity,0.01,0.00000,0.01,881,0.00,United States,NASDAQ,USD,1,--
1,--,ESC GCI LIBERTY INC SR ESCROW,Communication,Equity,0.28,0.00000,0.28,28033,0.00,United States,NASDAQ,USD,1,--
2,--,OMNIAB INC $15.00 VESTING Prvt,Health Care,Equity,0.02,0.00000,0.02,1502,0.00,United States,NO MARKET (E.G. UNLISTED),USD,1,--
3,--,OMNIAB INC $12.50 VESTING Prvt,Health Care,Equity,0.02,0.00000,0.02,1502,0.00,United States,NO MARKET (E.G. UNLISTED),USD,1,--
4,A,AGILENT TECHNOLOGIES INC,Health Care,Equity,10674454.32,0.06804,10674454.32,79494,134.28,United States,New York Stock Exchange Inc.,USD,1,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2678,ZUO,ZUORA INC CLASS A,Information Technology,Equity,426402.90,0.00272,426402.90,43071,9.90,United States,New York Stock Exchange Inc.,USD,1,--
2679,ZVRA,ZEVRA THERAPEUTICS INC,Health Care,Equity,162158.50,0.00103,162158.50,18725,8.66,United States,NASDAQ,USD,1,--
2680,ZWS,ZURN ELKAY WATER SOLUTIONS CORP,Industrials,Equity,1548221.85,0.00987,1548221.85,39285,39.41,United States,New York Stock Exchange Inc.,USD,1,--
2681,ZYME,ZYMEWORKS INC,Health Care,Equity,243538.71,0.00155,243538.71,14151,17.21,United States,NASDAQ,USD,1,--


In [29]:
returns = {}

for ticker in r3000.Ticker:
    
    if ticker == '--':
        continue
    else:
        sub_data = data.loc[data['TICKER']==ticker, 'RET']
        
        # Convert sub_data to float and drop NaN and 'C' values
        sub_data = sub_data[1:].dropna()
        sub_data = sub_data[sub_data != 'C']
        
        # Convert the remaining values to float
        sub_data = sub_data.astype(float)
        
        # Perform calculations or analysis on sub_data here
        if sub_data.empty:
            print(f"No data found for ticker: {ticker}")
        else:
            # Calculate cumulative return
            cumulative_return = (1 + sub_data).prod() - 1
            print(f"Ticker: {ticker}, Cumulative Return: {cumulative_return}")
            returns[ticker] = cumulative_return

Ticker: A, Cumulative Return: 2.7182490370218457
Ticker: AA, Cumulative Return: 0.29676059122199816
Ticker: AAL, Cumulative Return: -0.42519528401016005
Ticker: AAOI, Cumulative Return: 0.3205742128797626
Ticker: AAON, Cumulative Return: 4.632926163361383
Ticker: AAP, Cumulative Return: -0.391738527414495
Ticker: AAPL, Cumulative Return: 10.116864135925649
Ticker: AAT, Cumulative Return: -0.006314122675347367
Ticker: ABBV, Cumulative Return: 3.4965792847864066
Ticker: ABCB, Cumulative Return: 1.8519630210896803
Ticker: ABG, Cumulative Return: 3.167730235365595
Ticker: ABM, Cumulative Return: 0.9274753286015445
Ticker: ABNB, Cumulative Return: -0.059210364546907646
Ticker: ABR, Cumulative Return: 4.742428965449019
Ticker: ABSI, Cumulative Return: -0.8054646538036427
Ticker: ABT, Cumulative Return: 2.4910699000064125
Ticker: ABUS, Cumulative Return: -0.7619038633515933
Ticker: ACA, Cumulative Return: 2.0692185711363473
Ticker: ACAD, Cumulative Return: 0.24492163327137018
Ticker: ACCD, Cu

In [34]:
# Convert the dictionary to a sorted list of tuples (ticker, return)
sorted_returns = sorted(returns.items(), key=lambda x: x[1], reverse=True)
sorted_returns

[('NVDA', 131.36658029797582),
 ('CHE', 64.50923988757485),
 ('AGM', 58.803621577246844),
 ('BHVN', 43.73069965544118),
 ('HEI', 41.20343514665843),
 ('CELH', 39.50484814842949),
 ('AMD', 36.31980864083767),
 ('WSO', 35.940441710198066),
 ('OLED', 32.597896947435856),
 ('MCO', 30.242449146577414),
 ('AVGO', 26.497396438016885),
 ('MATX', 25.587975880527882),
 ('USD', 25.234434400047334),
 ('TSLA', 23.831024397644608),
 ('TTD', 22.907034316222237),
 ('MDGL', 21.79637251602447),
 ('BLDR', 21.169780801793856),
 ('RDNT', 20.462763295091936),
 ('GLOB', 20.153754659356384),
 ('MPWR', 19.357461482974696),
 ('MGPI', 18.867451335281796),
 ('ENPH', 18.49041900182138),
 ('CDNS', 18.4407960476817),
 ('HUBS', 18.287550441826973),
 ('SAVA', 18.232986735977903),
 ('FICO', 17.964406803785092),
 ('STZ', 17.833359859280094),
 ('KNSL', 17.770178741683637),
 ('CECO', 17.730385388390758),
 ('LEN', 17.307542514528603),
 ('ANET', 16.127946780109465),
 ('COOP', 15.92335470349321),
 ('LRCX', 15.737429740992873

In [49]:
# Extract the top 25% and bottom 25%
n = len(sorted_returns)
top_25_ret = [ret for _, ret in sorted_returns[:n // 4]]
bottom_25_ret = [ret for _, ret in sorted_returns[-n // 4:]]

### Pearson Correlation Coefficient

In [53]:
corr = stats.pearsonr(top_25_ret, bottom_25_ret)
corr

PearsonRResult(statistic=0.5321633457929187, pvalue=4.237562105450146e-49)

## Sharpe Ratio (top 25% are winners and last 25% are losers, suppose annual risk-free rate is 3%)

In [82]:
rf = 0.03/252
ratios = {}

for ticker in r3000.Ticker:
    
    if ticker == '--':
        continue
    else:
        sub_data = data.loc[data['TICKER']==ticker, 'RET']
        
        # Convert sub_data to float and drop NaN and 'C' values
        sub_data = sub_data[1:].dropna()
        sub_data = sub_data[sub_data != 'C']
        
        # Convert the remaining values to float
        sub_data = sub_data.astype(float)
        
        # Perform calculations or analysis on sub_data here
        if sub_data.empty:
            print(f"No data found for ticker: {ticker}")
        else:
            # Calculate sharpe ratio
            sharpe_ratio = (sub_data.mean()- rf)/sub_data.std()
            print(f"Ticker: {ticker}, sharpe ratio: {sharpe_ratio}")
            ratios[ticker] = sharpe_ratio

Ticker: A, sharpe ratio: 0.0327002749139943
Ticker: AA, sharpe ratio: 0.014373254517582217
Ticker: AAL, sharpe ratio: 0.004892529599923104
Ticker: AAOI, sharpe ratio: 0.02471923995524544
Ticker: AAON, sharpe ratio: 0.037223463595667514
Ticker: AAP, sharpe ratio: -0.0026957897614069168
Ticker: AAPL, sharpe ratio: 0.055873355564407284
Ticker: AAT, sharpe ratio: 0.0029864789410935628
Ticker: ABBV, sharpe ratio: 0.03692478645797407
Ticker: ABCB, sharpe ratio: 0.024477644891196665
Ticker: ABG, sharpe ratio: 0.03031970447159549
Ticker: ABM, sharpe ratio: 0.017149947749492113
Ticker: ABNB, sharpe ratio: 0.010730136913463436
Ticker: ABR, sharpe ratio: 0.03641966127257719
Ticker: ABSI, sharpe ratio: -0.013592506840566817
Ticker: ABT, sharpe ratio: 0.03303484208622701
Ticker: ABUS, sharpe ratio: 0.012371483729628319
Ticker: ACA, sharpe ratio: 0.04129026599790333
Ticker: ACAD, sharpe ratio: 0.019726982960973305
Ticker: ACCD, sharpe ratio: 0.0046379806154206605
Ticker: ACCO, sharpe ratio: 0.010013

In [84]:
sorted_ratios = sorted(ratios.items(), key=lambda x: x[1], reverse=True)
sorted_ratios

[('BYON', 0.34537021183504785),
 ('TRML', 0.3177600876655866),
 ('NLOP', 0.26788678131581223),
 ('CRGX', 0.25561585876371523),
 ('KRRO', 0.2516800672835723),
 ('KNF', 0.1805842642713538),
 ('IRON', 0.13409300111135047),
 ('BIRK', 0.13272043370012848),
 ('KGS', 0.11862560834533867),
 ('DRS', 0.10227006513133932),
 ('CXT', 0.09828367357016822),
 ('MBC', 0.0980775330219736),
 ('BCAL', 0.08597712330220414),
 ('CLMB', 0.08436038054891014),
 ('CEG', 0.07818577282231406),
 ('ASO', 0.07692689436523083),
 ('CHRD', 0.0747643266024058),
 ('APGE', 0.07408902332069206),
 ('ACLX', 0.07321641412775257),
 ('ESAB', 0.07262189543919377),
 ('KNSL', 0.07239187162917549),
 ('CELH', 0.06911809487967925),
 ('BHVN', 0.06910745279464206),
 ('CARR', 0.06872464649141038),
 ('ADEA', 0.06850742855100374),
 ('AVGO', 0.06570437398207962),
 ('CDNS', 0.06512497805613454),
 ('APG', 0.0649645500439955),
 ('COR', 0.0646885585745732),
 ('LLY', 0.06401485466492067),
 ('FBIN', 0.0623339784553478),
 ('HWM', 0.062311614975588

In [86]:
# Extract the top 25% and bottom 25%
n = len(sorted_ratios)
top_25_ratio = [ratio for _, ratio in sorted_ratios[:n // 4]]
bottom_25_ratio = [ratio for _, ratio in sorted_ratios[-n // 4:]]

### Pearson Correlation Coefficient

In [90]:
corr = stats.pearsonr(top_25_ratio, bottom_25_ratio)
corr

PearsonRResult(statistic=0.3819329266334707, pvalue=3.847832309986109e-24)

## Reference

In [28]:
tickers = ''

for ticker in r3000.Ticker[1800:]:
    
    if ticker == '--':
        continue
    else:
        tickers += ticker
        tickers += ' '
        
tickers = tickers.rstrip()
tickers

'OPRX ORA ORC ORCL ORGO ORI ORIC ORLY ORRF OSBC OSCR OSIS OSK OSPN OSUR OSW OTIS OTLK OTTR OUST OUT OVV OWL OXM OXY OZK P5N994 PACB PACK PACS PAG PAGS PAHC PANL PANW PAR PARA PARAA PARR PATH PATK PAX PAYC PAYO PAYS PAYX PB PBF PBH PBPB PCAR PCG PCH PCOR PCRX PCT PCTY PCVX PD PDCO PDFS PDM PEB PEBO PECO PEG PEGA PEN PENG PENN PEP PEPG PFBC PFC PFE PFG PFGC PFIS PFMT PFS PFSI PG PGC PGEN PGNY PGR PGRE PGY PH PHAT PHIN PHM PHR PI PII PINC PINS PIPR PJT PK PKE PKG PKST PL PLAB PLAY PLD PLL PLMR PLNT PLOW PLPC PLRX PLSE PLTK PLTR PLUG PLUS PLXS PLYM PM PMT PNC PNFP PNR PNTG PNW PODD POOL POR POST POWI POWL POWW PPBI PPC PPG PPL PPTA PR PRA PRAA PRAX PRCT PRDO PRG PRGO PRGS PRI PRIM PRK PRKS PRLB PRM PRMB PRME PRO PROK PRTA PRU PRVA PSA PSFE PSMT PSN PSTG PSTL PSX PTC PTCT PTEN PTGX PTLO PTON PTVE PUBM PUMP PVH PWP PWR PX PYCR PYPL PZZA QCOM QCRH QDEL QGEN QLYS QNST QRVO QS QSI QTRX QTTB QTWO R RAMP RAPT RARE RBA RBB RBBN RBC RBCAA RBLX RC RCEL RCKT RCL RCM RCUS RDFN RDN RDNT RDUS RDVT REAX 